# 04 - Data Export & Validation

Final notebook to validate the enriched data and export for PostgreSQL ingestion.

## Key Enhancements
- Schema alignment with `ingest_all_data.py` expected format
- Proper `features_metadata` JSON structure for database
- PRE-MATCH vs POST-MATCH field validation
- Data quality checks with detailed reporting
- Optimized CSV export for large datasets

## Objectives
1. Validate data quality and feature coverage
2. Check for data leakage (no future data in PRE-MATCH features)
3. Export final Parquet files
4. Generate PostgreSQL-ready CSVs aligned with `ingest_all_data.py`
5. Document final schema

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
from datetime import datetime

DATA_DIR = Path('../data')
PROCESSED_DIR = DATA_DIR / 'processed'

print("📊 Data Export & Validation Pipeline")
print("=" * 50)

📊 Data Export & Validation Pipeline


In [3]:
# Load enriched matches
matches = pd.read_parquet(PROCESSED_DIR / 'matches_enriched.parquet')
matches['date'] = pd.to_datetime(matches['date'])

print(f"✅ Loaded {len(matches):,} matches")
print(f"   Columns: {len(matches.columns)}")
print(f"   Memory: {matches.memory_usage(deep=True).sum() / 1024 / 1024:.2f} MB")

✅ Loaded 57,870 matches
   Columns: 107
   Memory: 53.87 MB


## 1. Data Quality Validation

In [4]:
def validate_data_quality(df: pd.DataFrame) -> dict:
    """Comprehensive data quality validation."""
    report = {
        'total_rows': len(df),
        'total_columns': len(df.columns),
        'null_summary': {},
        'coverage_by_tier': {},
        'issues': []
    }
    
    # Null analysis
    null_counts = df.isnull().sum()
    null_pct = (null_counts / len(df) * 100).round(2)
    
    for col in df.columns:
        if null_counts[col] > 0:
            report['null_summary'][col] = {
                'count': int(null_counts[col]),
                'pct': float(null_pct[col])
            }
    
    # Coverage by tier
    if 'tier' in df.columns:
        for tier in [1, 2, 3]:
            tier_df = df[df['tier'] == tier]
            if len(tier_df) > 0:
                # Check form feature coverage
                form_cols = [c for c in df.columns if 'form_' in c]
                if form_cols:
                    coverage = tier_df[form_cols[0]].notna().mean() * 100
                else:
                    coverage = 0
                report['coverage_by_tier'][f'tier_{tier}'] = {
                    'matches': len(tier_df),
                    'form_coverage_pct': round(coverage, 1)
                }
    
    return report

quality_report = validate_data_quality(matches)
print("📊 Data Quality Report")
print("=" * 50)
print(f"Total matches: {quality_report['total_rows']:,}")
print(f"Total columns: {quality_report['total_columns']}")
print(f"\nColumns with nulls: {len(quality_report['null_summary'])}")
print(f"\nCoverage by Tier:")
for tier, stats in quality_report['coverage_by_tier'].items():
    print(f"  {tier}: {stats['matches']:,} matches, {stats['form_coverage_pct']}% form coverage")

📊 Data Quality Report
Total matches: 57,870
Total columns: 107

Columns with nulls: 55

Coverage by Tier:
  tier_1: 2,472 matches, 99.6% form coverage
  tier_2: 28,485 matches, 98.6% form coverage
  tier_3: 26,405 matches, 94.4% form coverage


## 2. Point-in-Time Validation (No Data Leakage)

Verify that PRE-MATCH features don't use future match data.

In [5]:
def validate_no_leakage(df: pd.DataFrame, sample_size: int = 100) -> bool:
    """
    Spot check that form features don't include future data.
    For PRE-MATCH features, verify they only use past match data.
    """
    print("🔍 Validating point-in-time correctness...")
    
    # Check that early matches have null form features (teams with < min_periods)
    df_sorted = df.sort_values('date')
    
    # First matches for each team should have some null form values
    form_cols = [c for c in df.columns if 'form_' in c and not c.startswith('h2h')]
    
    if not form_cols:
        print("   ⚠️ No form columns found to validate")
        return True
    
    # Sample early matches
    early_matches = df_sorted.head(1000)
    null_rate = early_matches[form_cols].isna().mean().mean()
    
    print(f"   Early matches null rate: {null_rate*100:.1f}% (expected: >0 for teams with few games)")
    
    # Check that form values are reasonable
    issues = 0
    
    # Form wins/points should be <= n_games
    for n in [5, 10]:
        win_col = f'home_form_wins_{n}'
        if win_col in df.columns:
            max_wins = df[win_col].max()
            if max_wins > n:
                print(f"   ⚠️ {win_col} has max {max_wins} > {n}")
                issues += 1
    
    if issues == 0:
        print("✅ No obvious data leakage detected")
        return True
    else:
        print(f"⚠️ Found {issues} potential issues")
        return False

validate_no_leakage(matches)

🔍 Validating point-in-time correctness...
   Early matches null rate: 35.9% (expected: >0 for teams with few games)
✅ No obvious data leakage detected


True

## 3. Define Final Schema (Aligned with ingest_all_data.py)

In [6]:
# Define schema aligned with ingest_all_data.py features_metadata structure
# These are the fields that will be stored in the JSONB column

FEATURES_METADATA_SCHEMA = {
    # Ground truth outcomes (POST-MATCH)
    'outcomes': [
        'total_goals', 'over_1_5', 'over_2_5', 'over_3_5', 'btts',
        'home_clean_sheet', 'away_clean_sheet'
    ],
    
    # Match stats (POST-MATCH)
    'match_stats': [
        'home_possessionPct', 'home_totalShots', 'home_shotsOnTarget', 'home_wonCorners',
        'away_possessionPct', 'away_totalShots', 'away_shotsOnTarget', 'away_wonCorners'
    ],
    
    # Form features (PRE-MATCH) - 5 games
    'home_form_5': [
        'home_form_wins_5', 'home_form_draws_5', 'home_form_losses_5',
        'home_form_points_5', 'home_form_goals_scored_5', 'home_form_goals_conceded_5',
        'home_form_clean_sheets_5'
    ],
    'away_form_5': [
        'away_form_wins_5', 'away_form_draws_5', 'away_form_losses_5',
        'away_form_points_5', 'away_form_goals_scored_5', 'away_form_goals_conceded_5',
        'away_form_clean_sheets_5'
    ],
    
    # Form features (PRE-MATCH) - 10 games
    'home_form_10': [
        'home_form_wins_10', 'home_form_draws_10', 'home_form_losses_10',
        'home_form_points_10', 'home_form_goals_scored_10', 'home_form_goals_conceded_10',
        'home_form_clean_sheets_10'
    ],
    'away_form_10': [
        'away_form_wins_10', 'away_form_draws_10', 'away_form_losses_10',
        'away_form_points_10', 'away_form_goals_scored_10', 'away_form_goals_conceded_10',
        'away_form_clean_sheets_10'
    ]
}

# Flatten to get all feature columns
all_feature_cols = []
for category, cols in FEATURES_METADATA_SCHEMA.items():
    all_feature_cols.extend(cols)

print(f"📋 Features metadata schema: {len(all_feature_cols)} fields")
for category, cols in FEATURES_METADATA_SCHEMA.items():
    print(f"   {category}: {len(cols)} fields")

📋 Features metadata schema: 43 fields
   outcomes: 7 fields
   match_stats: 8 fields
   home_form_5: 7 fields
   away_form_5: 7 fields
   home_form_10: 7 fields
   away_form_10: 7 fields


## 4. Prepare PostgreSQL Export

Format data to match `ingest_all_data.py` expected structure.

In [7]:
def prepare_postgres_export(df: pd.DataFrame) -> pd.DataFrame:
    """
    Prepare DataFrame for PostgreSQL export.
    Matches the expected format in ingest_all_data.py import_processed_matches().
    """
    # Core columns expected by ingest script
    core_cols = ['eventId', 'homeTeamScore', 'awayTeamScore']
    
    # Feature columns to include in features_metadata
    feature_cols = [
        # Outcomes
        'total_goals', 'over_1_5', 'over_2_5', 'over_3_5', 'btts',
        'home_clean_sheet', 'away_clean_sheet',
        
        # Match stats
        'home_possessionPct', 'home_totalShots', 'home_shotsOnTarget', 'home_wonCorners',
        'away_possessionPct', 'away_totalShots', 'away_shotsOnTarget', 'away_wonCorners',
        
        # Form 5
        'home_form_wins_5', 'home_form_draws_5', 'home_form_losses_5',
        'home_form_points_5', 'home_form_goals_scored_5', 'home_form_goals_conceded_5',
        'home_form_clean_sheets_5',
        'away_form_wins_5', 'away_form_draws_5', 'away_form_losses_5',
        'away_form_points_5', 'away_form_goals_scored_5', 'away_form_goals_conceded_5',
        'away_form_clean_sheets_5',
        
        # Form 10
        'home_form_wins_10', 'home_form_draws_10', 'home_form_losses_10',
        'home_form_points_10', 'home_form_goals_scored_10', 'home_form_goals_conceded_10',
        'home_form_clean_sheets_10',
        'away_form_wins_10', 'away_form_draws_10', 'away_form_losses_10',
        'away_form_points_10', 'away_form_goals_scored_10', 'away_form_goals_conceded_10',
        'away_form_clean_sheets_10'
    ]
    
    # Filter to available feature columns
    available_features = [c for c in feature_cols if c in df.columns]
    
    # Select all columns for export
    export_cols = core_cols + available_features + [
        c for c in df.columns if c not in core_cols + available_features
    ]
    export_cols = [c for c in export_cols if c in df.columns]
    
    result = df[export_cols].copy()
    
    print(f"   Core columns: {len(core_cols)}")
    print(f"   Feature columns: {len(available_features)}")
    print(f"   Total columns: {len(export_cols)}")
    
    return result

print("Preparing PostgreSQL export...")
postgres_df = prepare_postgres_export(matches)

Preparing PostgreSQL export...
   Core columns: 3
   Feature columns: 43
   Total columns: 107


## 5. Export Final Files

In [8]:
# Select final columns for Parquet export
FINAL_SCHEMA = {
    'identifiers': ['eventId', 'date', 'leagueId', 'league_code', 'league_name', 'tier'],
    'teams': ['homeTeamId', 'home_team_name', 'awayTeamId', 'away_team_name'],
    'scores': ['homeTeamScore', 'awayTeamScore'],
    'outcomes': ['result', 'total_goals', 'over_1_5', 'over_2_5', 'over_3_5', 'btts',
                 'home_clean_sheet', 'away_clean_sheet'],
    'match_stats': ['home_possessionPct', 'home_totalShots', 'home_shotsOnTarget', 
                    'home_wonCorners', 'away_possessionPct', 'away_totalShots',
                    'away_shotsOnTarget', 'away_wonCorners']
}

# Get all available columns
available_cols = []
for category, cols in FINAL_SCHEMA.items():
    for col in cols:
        if col in matches.columns:
            available_cols.append(col)

# Add form features
form_cols = sorted([c for c in matches.columns if 'form_' in c or '_avg_' in c])
available_cols.extend(form_cols)

# Remove duplicates while preserving order
seen = set()
final_cols = []
for col in available_cols:
    if col not in seen:
        seen.add(col)
        final_cols.append(col)

print(f"📋 Final schema: {len(final_cols)} columns")
print(f"   Form features: {len(form_cols)}")

📋 Final schema: 78 columns
   Form features: 50


In [9]:
# Select final columns
final_matches = matches[[c for c in final_cols if c in matches.columns]].copy()

# Export Parquet (optimized for analysis)
parquet_path = PROCESSED_DIR / 'matches_final.parquet'
final_matches.to_parquet(parquet_path, index=False, compression='snappy')
print(f"✅ Exported {parquet_path}")
print(f"   Size: {parquet_path.stat().st_size / 1024 / 1024:.2f} MB")

# Export CSV for PostgreSQL - use postgres_df which has all features
csv_path = PROCESSED_DIR / 'matches_for_postgres.csv'
postgres_df.to_csv(csv_path, index=False)
print(f"✅ Exported {csv_path}")
print(f"   Size: {csv_path.stat().st_size / 1024 / 1024:.2f} MB")

✅ Exported ../data/processed/matches_final.parquet
   Size: 3.59 MB
✅ Exported ../data/processed/matches_for_postgres.csv
   Size: 31.98 MB


## 6. Generate Schema Documentation

In [10]:
# Generate comprehensive schema documentation
schema_doc = {
    'generated_at': datetime.now().isoformat(),
    'total_matches': len(final_matches),
    'date_range': {
        'min': str(final_matches['date'].min()),
        'max': str(final_matches['date'].max())
    },
    'columns': {},
    'tier_distribution': final_matches['tier'].value_counts().to_dict() if 'tier' in final_matches.columns else {},
    'features_metadata_schema': FEATURES_METADATA_SCHEMA,
    'pre_match_fields': [
        c for c in final_matches.columns if 'form_' in c or '_avg_' in c or 'h2h_' in c
    ],
    'post_match_fields': [
        'homeTeamScore', 'awayTeamScore', 'result', 'total_goals',
        'over_1_5', 'over_2_5', 'over_3_5', 'btts',
        'home_clean_sheet', 'away_clean_sheet',
        'home_possessionPct', 'home_totalShots', 'home_shotsOnTarget', 'home_wonCorners',
        'away_possessionPct', 'away_totalShots', 'away_shotsOnTarget', 'away_wonCorners'
    ]
}

for col in final_matches.columns:
    col_data = final_matches[col]
    schema_doc['columns'][col] = {
        'dtype': str(col_data.dtype),
        'null_count': int(col_data.isnull().sum()),
        'sample_values': col_data.dropna().head(3).tolist()[:3] if len(col_data.dropna()) > 0 else []
    }

# Save schema
schema_path = PROCESSED_DIR / 'schema_documentation.json'
with open(schema_path, 'w') as f:
    json.dump(schema_doc, f, indent=2, default=str)
print(f"✅ Saved schema documentation to {schema_path}")

✅ Saved schema documentation to ../data/processed/schema_documentation.json


In [11]:
# Save data quality report
quality_report['generated_at'] = datetime.now().isoformat()
quality_path = PROCESSED_DIR / 'data_quality_report.json'
with open(quality_path, 'w') as f:
    json.dump(quality_report, f, indent=2, default=str)
print(f"✅ Saved quality report to {quality_path}")

✅ Saved quality report to ../data/processed/data_quality_report.json


## 7. Summary Statistics

In [12]:
print("\n" + "=" * 60)
print("📊 FINAL DATA SUMMARY")
print("=" * 60)
print(f"\nTotal matches: {len(final_matches):,}")
print(f"Date range: {final_matches['date'].min().date()} to {final_matches['date'].max().date()}")
print(f"Columns: {len(final_matches.columns)}")

if 'tier' in final_matches.columns:
    print(f"\nMatches by Tier:")
    for tier in sorted(final_matches['tier'].dropna().unique()):
        count = len(final_matches[final_matches['tier'] == tier])
        print(f"  Tier {int(tier)}: {count:,} ({count/len(final_matches)*100:.1f}%)")

if 'result' in final_matches.columns:
    print(f"\nResult Distribution:")
    for result, count in final_matches['result'].value_counts().items():
        print(f"  {result}: {count:,} ({count/len(final_matches)*100:.1f}%)")

if 'over_2_5' in final_matches.columns:
    print(f"\nOver 2.5 Goals: {final_matches['over_2_5'].mean()*100:.1f}%")
if 'btts' in final_matches.columns:
    print(f"BTTS: {final_matches['btts'].mean()*100:.1f}%")

# PRE-MATCH feature summary
pre_match = [c for c in final_matches.columns if 'form_' in c or '_avg_' in c]
post_match = ['homeTeamScore', 'awayTeamScore', 'result', 'total_goals', 'btts']
print(f"\nPRE-MATCH features: {len(pre_match)}")
print(f"POST-MATCH fields: {len([c for c in post_match if c in final_matches.columns])}")

print("\n✅ Data preparation complete!")
print(f"   Output files in: {PROCESSED_DIR}")


📊 FINAL DATA SUMMARY

Total matches: 57,870
Date range: 2024-01-01 to 2025-12-15
Columns: 78

Matches by Tier:
  Tier 1: 2,472 (4.3%)
  Tier 2: 28,485 (49.2%)
  Tier 3: 26,405 (45.6%)

Result Distribution:
  H: 26,113 (45.1%)
  A: 17,271 (29.8%)
  D: 14,486 (25.0%)

Over 2.5 Goals: 49.3%
BTTS: 49.3%

PRE-MATCH features: 50
POST-MATCH fields: 5

✅ Data preparation complete!
   Output files in: ../data/processed


## Output Files

### Generated Files:
- `matches_final.parquet` - Main backtesting dataset (Parquet format)
- `matches_for_postgres.csv` - PostgreSQL import ready (aligned with `ingest_all_data.py`)
- `schema_documentation.json` - Column definitions, stats, and PRE/POST-MATCH classification
- `data_quality_report.json` - Data quality metrics
- `team_history.parquet` - Team match history with rolling features
- `leagues_clean.parquet` - League metadata with tiers

### Next Steps:
1. **Ingest to PostgreSQL**: Run `poetry run python backend/scripts/ingest_all_data.py`
2. **Backtesting**: Load `matches_final.parquet` for filter evaluation
3. **Filter Creation**: Use PRE-MATCH features only for filter conditions

### Important Notes:
- **PRE-MATCH features** (form, H2H, rolling stats): Safe for filter conditions
- **POST-MATCH features** (scores, actual stats): Ground truth for backtesting evaluation only
- Using POST-MATCH features in filter conditions causes **look-ahead bias**